In [1]:
from pymongo import MongoClient
from functools import reduce
import pprint
import time

### ROTINA PARA PREPARANÇÃO DOS DADOS

ROTA:
licitacao/:idLicitacao

OBJETIVO: Acrescenta o campo item nos documentos da coleção licitacao, a partir dos dados 
da coleção licitacaoVencedor.

PRÉ-CONDIÇÃO: Existência das coleções 'licitacao' e 'licitacaoVencedor'

PÓS CONDIÇÃO: 
Existência do atributo Item do tipo list nos documentos da coleção 'licitacao'.

MODELO DE DADOS:

    {
        cdIBGE : <string>,
        nmMunicipio : <string>,
        nmEntidade : <string>,
        idEntidade : <string>,
        idLicitacao :<string>,
        dsModalidade : <string>,
        nrLicitacao : <string>,
        nrAnoLicitacao : <string>,
        dtEdital : <ISODate>,
        dtAbertura : <ISODate>,
        vlLicitacao : <float>,
        dsNaturezaLicitacao : <string>,
        dsAvaliacaoLicitacao : <string>,
        dsClassificacaoObjeto : <string>,
        dsRegimeExecucao : <string>,
        dsObjeto : <string>,
        dsClausulaProrrogacao : <string>,
        item : <array>,
        vlTotalAdquiridoLicitacao : <float>
    }

Sendo o atributo item com os seguintes documentos

    item : [
        {
            'nmFornecedor' : <string>,
            'nrDocumentoFornecedor' : <string>,
            'nrLote' : <string>,
            'nrItem' : <string>,
            'nrQuantidade' : <double>,
            'dsUnidadeMedida' : <string>,
            'vlMinimoUnitario' : <double>,
            'vlMinimoTotal' : <double>,
            'vlMaximoUnitario': <double>,
            'vlMaximoTotal' : <double>,
            'dsItem' : <string>,
            'dsFormaPagamento' : <string>,
            'dsTipoEntregaProduto': <string>,
            'nrQuantidadePropostaLicitacao' : 
            'vlPropostaItem' : <double>,
            'dtValidadeProposta' : <string>,
            'dtPrazoEntregaPropostaLicitacao' : <string>,
            'nrClassificacao' : <string>,
            'dtHomologacao' : <string>,
            'nrQuantidadeVencedor' : <double>,
            'vlUnitarioVencedor' : <double>,
            'vlTotalVencedor' : <double>
        }
    ]


In [2]:
client = MongoClient("mongodb://localhost:27017")
DB = client['dbOpenvizTest']
def get_db():
    return DB

In [3]:
def determinar_idEntidade_para_processamento():
    db = get_db()
    match = { "$match" : {'item' : {'$exists' : False }} }
    project = {"$project" : {'_id' : 0, 'idEntidade' : 1, 'idLicitacao': 1} }
    group = { "$group" :{ 
                "_id" : "$idEntidade",
                "idsLicitacoes" : {
                    "$push" : "$idLicitacao"
                }
            }}
    pipeline = [match, project, group]

    cursor = db.licitacao.aggregate(pipeline)
    conjunto = list(cursor)
    return conjunto

In [10]:
def agrupa_itens_das_licitacoes(idLicitacao):
    match = { '$match' : { 'idlicitacao' : idLicitacao} }
    project = { '$project' : {'cdIBGE' : 0,
                              'idEntidade' : 0,
                              'nmEntidade' : 0,
                              'nrAnoLicitacao' : 0,
                              'nrLicitacao': 0,
                              'dsModalidadeLicitacao': 0,
                              'idUnidadeMedida' : 0 ,
                              'idTipoEntregaProduto': 0,
                              'DataReferencia' : 0,
                              'ultimoEnvioSIMAMNesteExercicio' : 0 
                             }
              }
    group = { '$group' : {
        '_id' : '$idlicitacao',
        'item' : { '$push' : {
            'nmFornecedor' : '$nmFornecedor',
            'nrDocumento' : '$nrDocumento',
            'nrLote' :'$nrLote',
            'nrItem' : '$nrItem',
            'nrQuantidade' : '$nrQuantidade',
            'dsUnidadeMedida' : '$dsUnidadeMedida',
            'vlMinimoUnitario' : '$vlMinimoUnitarioItem',
            'vlMinimoTotal' : '$vlMinimoTotal',
            'vlMaximoUnitario': '$vlMaximoUnitarioitem',
            'vlMaximoTotal' : '$vlMaximoTotal',
            'dsItem' : '$dsItem',
            'dsFormaPagamento' : '$dsFormaPagamento',
            'dsTipoEntregaProduto': '$dsTipoEntregaProduto',
            'nrQuantidadePropostaLicitacao' : '$nrQuantidadePropostaLicitacao',
            'vlPropostaItem' : '$vlPropostaItem',
            'dtValidadeProposta' : '$dtValidadeProposta',
            'dtPrazoEntregaPropostaLicitacao' : '$dtPrazoEntregaPropostaLicitacao',
            'nrClassificacao' : '$nrClassificacao',
            'dtHomologacao' : '$dtHomologacao',
            'nrQuantidadeVencedor' : '$nrQuantidadeVencedorLicitacao',
            'vlUnitarioVencedor' : '$vlUnitarioVencedorLicitacao',
            'vlTotalVencedor' : '$vlTotalVencedorLicitacao'
        }},
        'vlTotalAdquiridoLicitacao' : {
            '$sum' : '$vlTotalVencedorLicitacao'
        }
    }}
    pipeline = [match, project, group]
    db = get_db()
    cursor =  db.licitacaoVencedor.aggregate(pipeline)
    return cursor


In [11]:
def update_field(cursor = None):
    if not cursor:
        raise AttributeError('parametro invalido em update_field')
    db = get_db()
    count = 0
    for doc in cursor:
        result = db.licitacao.update_one({'idLicitacao' : doc['_id']},
                                         { '$set' : {
                                             'item' : doc['item'],
                                             'vlTotalAdquiridoLicitacao': doc['vlTotalAdquiridoLicitacao']
                                         }})
        count += 1
    return count

In [17]:
def execute():
    db = get_db()
    lstLicitacaoesPorIdEntidade = determinar_idEntidade_para_processamento()
    for entidade in lstLicitacaoesPorIdEntidade:
        for idLicitacao in entidade['idsLicitacoes']:
            cursor = agrupa_itens_das_licitacoes(idLicitacao)
            nrDocs = update_field(cursor)
    return lstLicitacaoesPorIdEntidade

In [19]:
%time result = execute()

Wall time: 13.2 s


In [21]:
pprint.pprint(result)

[{'_id': '101235', 'idsLicitacoes': ['790976', '798160', '798589']},
 {'_id': '446696', 'idsLicitacoes': ['685666']},
 {'_id': '12354',
  'idsLicitacoes': ['712919',
                    '728169',
                    '739618',
                    '741444',
                    '741447',
                    '741452',
                    '741453',
                    '746637',
                    '746641',
                    '746642',
                    '759281',
                    '759282']},
 {'_id': '12259',
  'idsLicitacoes': ['689264',
                    '689265',
                    '689266',
                    '689272',
                    '689282',
                    '689284',
                    '689285',
                    '689287',
                    '689288',
                    '689289',
                    '689291',
                    '689292',
                    '689294',
                    '691157',
                    '691166',
                    '691167',
    